In [ ]:
%matplotlib inline

import sys
sys.path.append('../..')

from matplotlib import pylab as plt

import itertools
import numpy
import scipy
import scipy.special
import time
import random

from ipywidgets import interact
from IPython.display import display, Markdown, clear_output

from crocodile.simulate import simulate_point
from crocodile.antialias import *

First, some grid characteristics. Only theta is actually important here, the rest is just decides the range of the example $u/v$ values.

In [ ]:
theta = 0.1
lam = 18000
grid_size = int(theta * lam)

In [ ]:
def kernel_oversample(ff, Qpx, s=None, P = 1):
    """
    Takes a farfield pattern and creates an oversampled convolution
    function.

    If the far field size is smaller than N*Qpx, we will pad it. This
    essentially means we apply a sinc anti-aliasing kernel by default.

    :param ff: Far field pattern
    :param Qpx: Factor to oversample by -- there will be Qpx x Qpx convolution arl
    :param s: Size of convolution function to extract
    :returns: Numpy array of shape [ov, ou, v, u], e.g. with sub-pixel
      offsets as the outer coordinates.
    """

    # Pad the far field to the required pixel size
    N = ff.shape[0]
    if s is None: s = N
    padff = pad_mid(ff, N*Qpx*P)

    # Obtain oversampled uv-grid
    af = fft(padff)

    # Extract kernels
    return extract_oversampled(extract_mid(af, N*Qpx), Qpx, s)

Determine $u/v$ gridding function to use. Three choices here - trivial, Sze-Tan's version and PSWF. `x0` decides how much of the image coordinate space we can actually use without errors rising.

We use that to calculate the appropriate grid step length `du` for good accuracy in our target field of view `theta`:

In [ ]:
grid_size = 2047
aa_over = 256
aa_support = 10
aa_x0 = 0.375
aa_mode = 0
aa_szetan = False
aa_nifty = True
aa_parameter = numpy.pi*aa_support/2
if aa_support == 1:
    print("Using trivial gridder")
    aa_gcf = numpy.ones((aa_over, aa_support))
    def aa(x): return numpy.ones_like(x)
elif aa_nifty:
    print("Using exponential of semi-circle with beta=%d" % (aa_support))
    aa = numpy.exp(aa_parameter*(numpy.sqrt(1-(2*coordinates(grid_size))**2)-1))
    aa_gcf = kernel_oversample(aa, aa_over, aa_support) / grid_size
    def aa(x):
        return numpy.exp(aa_parameter*(numpy.sqrt(1-(2*x)**2)-1))
elif aa_szetan:
    print("Using Sze-Tan's gridder with R=%d, x_0=%g" % (aa_support//2, aa_x0))
    aa_gcf = sze_tan_gridder(aa_support//2, aa_x0, aa_over)
    def aa(x):
        return sze_tan_grid_correction_gen(aa_support//2, aa_x0, x)
    print("Mean error:", sze_tan_mean_error(aa_support//2, aa_x0))
else:
    print("Using PSWF with mode %d and parameter %g" % (aa_mode, aa_parameter))
    aa = scipy.special.pro_ang1(aa_mode, aa_mode, aa_parameter, 2*coordinates(grid_size))[0]
    aa_gcf = kernel_oversample(aa, aa_over, aa_support) / grid_size
    def aa(x):
        return scipy.special.pro_ang1(aa_mode, aa_mode, aa_parameter, 2*x)[0]
    
# Calculate appropriate step length to give us full accuracy for a field of view of size theta
du = du_opt = aa_x0/(theta/2)
print("Optimal du =", du)

# Plot gridding function
plt.rcParams['figure.figsize'] = 10, 5
r = numpy.arange(-aa_over*(aa_support//2), aa_over*((aa_support+1)//2)) / aa_over
plt.semilogy(du_opt*r, numpy.abs(numpy.transpose(aa_gcf).flatten()));
#plt.semilogy(du_opt*r, numpy.transpose(aa2_gcf).flatten().real);
plt.xticks(du_opt*numpy.arange(-(aa_support//2), ((aa_support+1)//2)+1))
plt.grid(True);plt.xlabel('u/v [$\lambda$]');plt.title('$u/v$ Gridder');plt.show()

# Plot grid correction function
theta_x0 = theta/aa_x0/2
x = coordinates(101)
plt.semilogy(theta*x/aa_x0/2, aa(x));
plt.title('$u/v$ Grid correction');plt.grid(True);plt.xlabel('l [1]')
plt.axvspan(theta/2, theta_x0/2, color='lightgray', hatch='x', alpha=0.5)
plt.axvspan(-theta/2, -theta_x0/2, color='lightgray', hatch='x', alpha=0.5)
plt.annotate('(unused)', xy=((theta+theta_x0)/4,0.9), ha='center', color='gray')
plt.annotate('(unused)', xy=(-(theta+theta_x0)/4,0.9), ha='center', color='gray');
#plt.semilogy(theta*coordinates(grid_size)/aa_x0/2, anti_aliasing_function(grid_size, aa_mode, aa_parameter));

In [ ]:
aa_support_w = 8
aa_x0_w = 0.125
aa_szetan_w = False
aa_nifty_w = False
aa_parameter_w = numpy.pi*aa_support_w/2
if aa_support_w == 1:
    print("Using trivial gridder")
    aa_gcf_w = numpy.ones((aa_over, aa_support_w))
    def aa_w(x): return numpy.ones_like(x)
elif aa_nifty_w:
    print("Using exponential of semi-circle with beta=%d" % (aa_support))
    aa_gcf_w = kernel_oversample(
        numpy.exp(aa_support*(numpy.sqrt(1-(2*coordinates(grid_size))**2)-1)),
        aa_over, aa_support) / grid_size
    def aa_w(x):
        return numpy.exp(aa_support*(numpy.sqrt(1-(2*x)**2)-1))
elif aa_szetan_w:
    print("Using Sze-Tan's gridder with R=%d, x_0=%g" % (aa_support_w//2, aa_x0_w))
    aa_gcf_w = sze_tan_gridder(aa_support_w//2, aa_x0_w, aa_over)
    def aa_w(x):
        return sze_tan_grid_correction_gen(aa_support_w//2, aa_x0_w, x)
    print("Mean error:", sze_tan_mean_error(aa_support_w//2, aa_x0_w))
else:
    aa_w = anti_aliasing_function(grid_size, 0, aa_parameter_w)
    aa_gcf_w = kernel_oversample(aa_w, aa_over, aa_support_w) / grid_size
    def aa_w(x):
        return scipy.special.pro_ang1(aa_mode, aa_mode, aa_parameter_w, 2*x)[0]

# Calculate appropriate step length to give us full accuracy for a field of view of size theta
max_n = 1.0 - numpy.sqrt(1.0 - 2*(theta/2)**2)
print("max_n =", max_n)
dw = dw_opt = aa_x0_w / max_n
print("Optimal dw =", dw)

# Plot gridding function
plt.rcParams['figure.figsize'] = 10, 5
r = numpy.arange(-aa_over*(aa_support_w//2), aa_over*((aa_support_w+1)//2)) / aa_over
plt.semilogy(dw_opt*r, numpy.transpose(aa_gcf_w).flatten().real);
plt.xticks(dw_opt*numpy.arange(-(aa_support_w//2), ((aa_support_w+1)//2)+1))
plt.grid(True); plt.xlabel('w [$\lambda$]'); plt.title('$w$ Gridder'); plt.show()

x = coordinates(101)
plt.semilogy(max_n*x/aa_x0_w, aa_w(x));
plt.title('$w$ Grid correction'); plt.grid(True); plt.xlabel('$n$ [1]');
max_n_x0 = max_n/aa_x0_w/2
plt.axvspan(max_n, max_n_x0, color='lightgray', hatch='x', alpha=0.5)
plt.axvspan(-max_n, -max_n_x0, color='lightgray', hatch='x', alpha=0.5)
plt.annotate('(unused)', xy=((max_n+max_n_x0)/2,0.9), ha='center', color='gray')
plt.annotate('(unused)', xy=(-(max_n+max_n_x0)/2,0.9), ha='center', color='gray');

Now generate some sources on the sky. We use a random pattern to make reasonably sure that we are not hand-picking a good sky pattern.

In [ ]:
Npt = 500
points = theta * (numpy.random.rand(Npt,2)-0.5)

#points = list(theta/10 * numpy.array(list(itertools.product(range(-5, 6), range(-5, 6)))))
#points.append((theta/3,0))
#points = numpy.array(points)

plt.rcParams['figure.figsize'] = 8, 8
plt.scatter(points[:,0], points[:,1]);

Set up code to predict visibilities - either directly or by visibilities weighted by the grid correction and offset in a grid-like fashion.

In [ ]:

def predict(dist_uvw, du=du_opt, dw=dw_opt, apply_aa = False, apply_aa_w = False):
    # Get image coordinates
    ls, ms = numpy.transpose(points)
    ns = numpy.sqrt(1.0 - ls**2 - ms**2) - 1
    # Evaluate grid correction functions in uv & w
    aas = numpy.ones(len(ls))
    if apply_aa:
        aas *= aa(du*ls) * aa(du*ms)
    if apply_aa_w:
        aas *= aa_w(dw*ns)
    # Now simulate points, dividing out grid correction
    vis = 0
    for l,m, a in zip(ls, ms, aas):
        vis += simulate_point(dist_uvw, l, m) / a
    return vis

def predict_grid(u,v,w,ov_u,ov_v,ov_w,du=du_opt, dw=dw_opt, visualise=False):
    
    # Generate offsets that we are going to sample at
    ius, ivs, iws = numpy.meshgrid(numpy.arange(aa_support), numpy.arange(aa_support), numpy.arange(aa_support_w))
    dus = du*(ius.flatten()-(aa_support//2)+ov_u/aa_over)
    dvs = du*(ivs.flatten()-(aa_support//2)+ov_v/aa_over)
    dws = dw*(iws.flatten()-(aa_support_w//2)+ov_w/aa_over)
    
    # Get grid convolution function for offsets
    aas = aa_gcf[ov_u,ius.flatten()] * aa_gcf[ov_v,ivs.flatten()] * aa_gcf_w[ov_w,iws.flatten()]

    # Add offsets to all uvw coordinates
    us = numpy.array(u)[:,numpy.newaxis] + dus[numpy.newaxis,:]
    vs = numpy.array(v)[:,numpy.newaxis] + dvs[numpy.newaxis,:]
    ws = numpy.array(w)[:,numpy.newaxis] + dws[numpy.newaxis,:]
    
    # Visualise sampling pattern?
    if visualise:
        ax = plt.subplot(111, projection='3d')
        ax.scatter(us,vs,ws, color='red');
        ax.set_xlabel('u'); ax.set_ylabel('v'); ax.set_zlabel('w')

    # Predict visibilities
    vis = predict(numpy.transpose([us.flatten(),vs.flatten(),ws.flatten()]),
                  du=du, dw=dw, apply_aa=True, apply_aa_w=True).reshape(us.shape)
    
    # Convolve with gridder, sum up
    return numpy.sum(vis * aas[numpy.newaxis,:], axis=1)


Now we can test the performance of the sampling over a wide variety of parameters. Note that `u`,`v` and `w` do not actually matter too much, but we get into trouble quickly by increasing `du` or `dw` -- that is when we start using our gridder for inaccurate image coordinates!

In [ ]:
@interact(u=(-lam/2,lam/2,0.1),v=(-lam/2,lam/2,0.1),w=(-lam/2,lam/2,0.1),
          ov_u=(0,aa_over-1), ov_v=(0,aa_over-1), ov_w=(0,aa_over-1),
          du=(du_opt/10,du_opt*2,du_opt/10), dw=(dw_opt/10,dw_opt*2,dw_opt/10))
def test(u=0,v=0,w=0, ov_u=0,ov_v=0,ov_w=0, du=du_opt, dw=dw_opt):
    vis = predict(numpy.transpose([[u],[v],[w]]))
    print("Direct: ", vis[0])
    vis_sum = predict_grid([u],[v],[w],ov_u,ov_v,ov_w,du,dw)
    print("Grid:   ", vis_sum[0])
    print("Error:  ", numpy.abs(vis[0]-vis_sum[0]) / numpy.sqrt(Npt))

We can make a quick statistic by feeding in a good couple of points:

In [ ]:
N = 500
us = lam * (numpy.random.rand(N)-0.5)
vs = lam * (numpy.random.rand(N)-0.5)
ws = lam * (numpy.random.rand(N)-0.5)
ov_u = random.randint(0, aa_over-1)
ov_v = random.randint(0, aa_over-1)
ov_w = random.randint(0, aa_over-1)
vis = predict(numpy.transpose([us,vs,ws]))
grid_vis = predict_grid(us,vs,ws,ov_u,ov_v,ov_w)
diff = numpy.abs(vis-grid_vis)
mean_err = numpy.sqrt(numpy.mean(diff**2)) / numpy.mean(numpy.abs(vis))
print("Mean error:", mean_err)